In [2]:
# Function to sentence-tokenise answers 

from nltk.tokenize import sent_tokenize
import pandas as pd

def sent_tokenise_answer(data, col_ind) :
    
    """ Function to sentence-tokenise answers. Return a list of lists that contain sentences as strings.
        
        data = name of the dataframe
        col_ind = index of the column that contains the texts to be sentence-tokenised
    """
    
    sents_collector = []
    
    for answer in data.iloc[:, col_ind] :
        
        #if no answer was provided, return empty string list
        if pd.isnull(answer) :
            sents_collector.append(list(""))
            
        # an answer was provided    
        else :
            sents_collector.append(sent_tokenize(answer))
            
    return(sents_collector)


In [3]:
cons1_df = pd.read_csv("/Users/alessia/Documents/DataScience/NLP_Project/Outputs/cons1_lemmas_df.csv", nrows=5)

In [4]:
# Get column index of questions
idx_Q1 = cons1_df.columns.get_loc(str([col for col in cons1_df if 'census methods' in str(col)][0]))

In [5]:
idx_Q1     #42

42

In [6]:
# Function to word-tokenise sentences 

from nltk.tokenize import word_tokenize

def word_tokenise_answer(data, col_ind) :
    
    """ Function to word-tokenise answers' sentences. Return a list of lists of words as strings. 
        Required input, a list of lists containing sentences as strings.
        
        data = name of the dataframe
        col_ind = index of the column that contains the list of sentences to be word-tokenised
    """
    
    sents_collector = []
    
    for answer in data.iloc[:, col_ind] :
        
        words_collector = []
        
        #if no answer was provided, return empty string list
        if not answer:
            sents_collector.append(list(""))
            
        # an answer was provided    
        else :
            for sent in answer :
                words_collector.append(word_tokenize(sent))
                
        sents_collector.append(words_collector)
            
    return(sents_collector)


In [7]:
[type(s) for s in sent_tokenise_answer(cons1_df, idx_Q1)[4]]

[str, str, str, str, str, str, str, str, str, str, str, str, str, str]

In [8]:
cons1_df.loc[:, 'test1'] = sent_tokenise_answer(cons1_df, idx_Q1)

In [9]:
test1_idx = cons1_df.columns.get_loc('test1')      #73

In [10]:
cons1_df.loc[:, 'test1']

0                                                   []
1                                                   []
2                                                   []
3    [Moving to a primarily online census: an inevi...
4    [A regular full population census is absolutel...
Name: test1, dtype: object

In [11]:
cons1_df.iloc[:, test1_idx]

0                                                   []
1                                                   []
2                                                   []
3    [Moving to a primarily online census: an inevi...
4    [A regular full population census is absolutel...
Name: test1, dtype: object

In [12]:
word_tokenise_answer(cons1_df, test1_idx)

[[],
 [],
 [],
 [],
 [],
 [],
 [['Moving',
   'to',
   'a',
   'primarily',
   'online',
   'census',
   ':',
   'an',
   'inevitable',
   'and',
   'necessary',
   'evolution',
   'of',
   'the',
   'existing',
   'approach',
   '.'],
  ['Admin',
   'data',
   'and',
   'surveys',
   ':',
   'an',
   'unknown',
   'quantity',
   ',',
   'dependent',
   'on',
   'the',
   'quality',
   'of',
   'admin',
   'data',
   ',',
   'and',
   'not',
   'clear',
   'how',
   'well',
   'it',
   'would',
   'fulfil',
   'the',
   'primary',
   'aim',
   'of',
   'a',
   'census',
   ':',
   'to',
   'produce',
   'an',
   'accurate',
   'and',
   'independent',
   'estimate',
   'of',
   'the',
   'size',
   'and',
   'composition',
   'of',
   'the',
   'population',
   '.']],
 [['A',
   'regular',
   'full',
   'population',
   'census',
   'is',
   'absolutely',
   'necessary',
   '.'],
  ['It',
   'is',
   'the',
   'only',
   'way',
   'to',
   'ensure',
   'that',
   'all',
   'population'

In [29]:
# Define function to calculate polarity score for the answers in our dataset

# import key modules
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

from numpy import nan
    

def get_sentiment_score(data, col_ind, score_type = 'compound') :
    """ 
    
    Calculate sentiment analysis score (score_type: 'compound' default, 'positive', 'negative')
    for the values in the specified dataframe column.
    
    Return a list of scores, one score for each sentence in the column cell.
    
    """
    
    # empty list collector of scores
    sentiment_bag = []
    
    for answer in data.iloc[:, col_ind] :
        
        #print(len(answer))
        
        # no answer was provided, return NA
        if not answer : 
            sentiment_bag.append(nan)
        
        # answer is made of only 1 sentence    
        elif len(answer) == 1 :
            sentiment_bag.append(analyser.polarity_scores(answer)[score_type])
        
        # answer contains more than one sentence
        elif len(answer) > 1 :
            sentiment_bag.append([analyser.polarity_scores(s)[score_type] for s in answer])
    
    return(sentiment_bag)
    

In [24]:
# test
get_sentiment_score(cons1_df, test1_idx, 'compound')

0
0
0
2
14


[nan,
 nan,
 nan,
 [0.0, -0.4585],
 [0.0,
  0.3818,
  0.0,
  0.4404,
  0.0,
  0.0,
  0.4404,
  0.0,
  0.8481,
  0.7964,
  -0.1779,
  0.0,
  0.4404,
  0.4404]]

In [36]:
# answer after sent-tokenisation, before word-tokenisation
[print(list(s)) for s in cons1_df.iloc[3:, test1_idx]]

['Moving to a primarily online census: an inevitable and necessary evolution of the existing approach.', 'Admin data and surveys: an unknown quantity, dependent on the quality of admin data, and not clear how well it would fulfil the primary aim of a census: to produce an accurate and independent estimate of the size and composition of the population.']
['A regular full population census is absolutely necessary.', 'It is the only way to ensure that all population and social statistical estimates and projections are grounded in reality.', 'Data for small areas is absolutely necessary.', 'Without it, local authorities would be unable to plan services, target resources and measure performance effectively.', 'The cost to the country would exceed the cost of a decennial census.', 'Would suggest that both options should be carried out, if possible, rather than one or the other.', 'That way, we get the accurate picture every ten years, and a good/ useful indication of trends throughout the in

[None, None]

In [121]:
[print(str(s)) for s in sent_tokenise_answer(cons1_df, idx_Q1)]

[]
[]
[]
['Moving to a primarily online census: an inevitable and necessary evolution of the existing approach.', 'Admin data and surveys: an unknown quantity, dependent on the quality of admin data, and not clear how well it would fulfil the primary aim of a census: to produce an accurate and independent estimate of the size and composition of the population.']
['A regular full population census is absolutely necessary.', 'It is the only way to ensure that all population and social statistical estimates and projections are grounded in reality.', 'Data for small areas is absolutely necessary.', 'Without it, local authorities would be unable to plan services, target resources and measure performance effectively.', 'The cost to the country would exceed the cost of a decennial census.', 'Would suggest that both options should be carried out, if possible, rather than one or the other.', 'That way, we get the accurate picture every ten years, and a good/ useful indication of trends througho

[None, None, None, None, None]

In [30]:
[print(result) for result in get_sentiment_score(cons1_df, test1_idx)]

nan
nan
nan
[0.0, -0.4585]
[0.0, 0.3818, 0.0, 0.4404, 0.0, 0.0, 0.4404, 0.0, 0.8481, 0.7964, -0.1779, 0.0, 0.4404, 0.4404]


[None, None, None, None, None]

In [37]:
import numpy as np
print([mean(np.array(result)) for result in get_sentiment_score(cons1_df, test1_idx)])
print([median(np.array(result)) for result in get_sentiment_score(cons1_df, test1_idx)])

[nan, nan, nan, -0.22925000000000001, 0.25785714285714284]
[nan, nan, nan, -0.22925000000000001, 0.19089999999999999]


/Users/alessia/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4016: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


In [45]:
print(median(np.array((0.0, 0.3818, 0.0, 0.4404, 0.0, 0.0, 0.4404, 0.0, 0.8481, 0.7964, -0.1779, 0.0, 0.4404, 0.4404))))
print(0.3818/2)

0.1909
0.1909


In [127]:
[print(analyser.polarity_scores(s)) for s in ["Admin data and surveys: an unknown quantity, dependent on the quality of admin data, and not clear how well it would fulfil the primary aim of a census: to produce an accurate and independent estimate of the size and composition of the population.", "The ability to do that would be severely hampered is detailed small area population data were not available."]]

{'neg': 0.091, 'neu': 0.909, 'pos': 0.0, 'compound': -0.4585}
{'neg': 0.141, 'neu': 0.751, 'pos': 0.108, 'compound': -0.1779}


[None, None]